In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

from keras.regularizers import l1, l2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend
from keras import optimizers

from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder

import random

Using TensorFlow backend.


In [2]:
# SEED = 10
# random.seed(SEED)
# np.random.seed(SEED)

In [3]:
# p = 0.4
# df = pd.concat([pd.read_csv('articles2.csv',skiprows=lambda i: 1>0 and random.random() > p),
#                 pd.read_csv('articles1.csv',skiprows=lambda i: 1>0 and random.random() > p)],
#                 axis=1)

In [4]:
df1 = pd.read_csv('articles1.csv')
df2 = pd.read_csv('articles2.csv')

In [5]:
df = pd.concat([df1,df2])

In [6]:
outlets = ['Breitbart','New York Times','Atlantic','New York Post']
bool_mask = df['publication'].isin(outlets)
df = df[bool_mask][['content','publication']]
df = df.dropna()
mask = np.random.rand(len(df)) < 0.2
trimmed = df[mask]

In [7]:
trimmed

,content,publication
2,"When Walt Disney’s “Bambi” opened in 1942, cri...",New York Times
7,"Danny Cahill stood, slightly dazed, in a blizz...",New York Times
11,"THOMPSONS, Tex. — Can one of the most promisin...",New York Times
15,Finally. The Second Avenue subway opened in Ne...,New York Times
24,WASHINGTON — The most powerful and ambitious C...,New York Times
...,...,...
48562,Two innocent bystanders were shot — including ...,New York Post
48565,"CARACAS, Venezuela — The mob didn’t know at fi...",New York Post
48576,Next time the Canadiens decide to party after ...,New York Post
48579,If Mayor Bill de Blasio personally sought dona...,New York Post


In [8]:
for c in trimmed['content']:
    for o in outlets:
        c.replace(o, "")

In [9]:
train_msk = np.random.rand(len(trimmed)) < 0.70
train = trimmed[train_msk]
test = trimmed[~train_msk]

In [10]:
xa_train = train['content'].to_numpy()
ya_train = train['publication'].to_numpy().reshape(-1, 1)
xa_test = test['content'].to_numpy()
ya_test = test['publication'].to_numpy().reshape(-1, 1)

In [11]:
def split_sentences(xa, ya, punctuation=". "):
    xs = []
    ys = []
    for xi,yi in zip(xa,ya):
        sentences = xi.split(punctuation)
        for s in sentences:
            xs.append(s)
            ys.append(yi)
    return np.array(xs), np.array(ys)

In [12]:
xs_train, ys_train = split_sentences(xa_train, ya_train)
xs_test, ys_test = split_sentences(xa_test, ya_test)

In [13]:
def filter_trigger(xs, ys, trigger_word="Trump"):
    xt = []
    yt = []
    for xi, yi in zip(xs, ys):
        if trigger_word in xi:
            xt.append(xi)
            yt.append(yi)
    return np.array(xt), np.array(yt)

In [14]:
xt_train, yt_train = filter_trigger(xs_train, ys_train)
xt_test, yt_test = filter_trigger(xs_test, ys_test)

In [15]:
len(xt_train)

19467

In [16]:
xt_train

array(['Trump? The technology, carbon capture, involves pulling carbon dioxide out of smokestacks and industrial processes before the gas can make its way into the atmosphere',
       'Trump’s denial of the overwhelming scientific evidence supporting climate change, a view shared by many of his cabinet nominees, might appear to doom any such environmental initiatives',
       'What the Trump administration will do with carbon capture is, at this point, anyone’s guess',
       ...,
       '“Most folks up here detest Trump and won’t vote for him, but dislike her intensely at the same time',
       'In the same message, he mocked the GOP nominee’s attempts to reach out to minority communities, writing, “Trump just looks stupid trying to appeal to blacks and Latinos',
       '” In another he blasted the birther movement, which Trump stoked by questioning where President Obama was born'],
      dtype='<U1159')

In [17]:
enc = OneHotEncoder()
Y_train = enc.fit_transform(yt_train).toarray()
Y_test = enc.fit_transform(yt_test).toarray()

In [18]:
# Load the encoder:
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  embed = hub.load("universal-sentence-encoder-large")
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session:
session = tf.Session(graph=g)
session.run(init_op)

In [19]:
#Function to compute all embeddings for each sentence:
#Be patient, takes a little while:
def similarity_matrix(merge_list):
    #initialize distance array:
    #initialize embeddings array:
    emb_all = np.zeros([len(merge_list),512])
    #Outer for loop:
    for i in range(len(merge_list)):
        print("iteration: {}/{}".format(i,len(merge_list)))
        #Here is where we run the previously started session, so it is important to run previous step succesfully:
        i_emb = session.run(embedded_text, feed_dict={text_input: [merge_list[i]]})
        emb_all[i,:] = i_emb
    return emb_all

In [20]:
X_train = similarity_matrix(xt_train)

iteration: 0/19467
iteration: 1/19467
iteration: 2/19467
iteration: 3/19467
iteration: 4/19467
iteration: 5/19467
iteration: 6/19467
iteration: 7/19467
iteration: 8/19467
iteration: 9/19467
iteration: 10/19467
iteration: 11/19467
iteration: 12/19467
iteration: 13/19467
iteration: 14/19467
iteration: 15/19467
iteration: 16/19467
iteration: 17/19467
iteration: 18/19467
iteration: 19/19467
iteration: 20/19467
iteration: 21/19467
iteration: 22/19467
iteration: 23/19467
iteration: 24/19467
iteration: 25/19467
iteration: 26/19467
iteration: 27/19467
iteration: 28/19467
iteration: 29/19467
iteration: 30/19467
iteration: 31/19467
iteration: 32/19467
iteration: 33/19467
iteration: 34/19467
iteration: 35/19467
iteration: 36/19467
iteration: 37/19467
iteration: 38/19467
iteration: 39/19467
iteration: 40/19467
iteration: 41/19467
iteration: 42/19467
iteration: 43/19467
iteration: 44/19467
iteration: 45/19467
iteration: 46/19467
iteration: 47/19467
iteration: 48/19467
iteration: 49/19467
iteration:

iteration: 397/19467
iteration: 398/19467
iteration: 399/19467
iteration: 400/19467
iteration: 401/19467
iteration: 402/19467
iteration: 403/19467
iteration: 404/19467
iteration: 405/19467
iteration: 406/19467
iteration: 407/19467
iteration: 408/19467
iteration: 409/19467
iteration: 410/19467
iteration: 411/19467
iteration: 412/19467
iteration: 413/19467
iteration: 414/19467
iteration: 415/19467
iteration: 416/19467
iteration: 417/19467
iteration: 418/19467
iteration: 419/19467
iteration: 420/19467
iteration: 421/19467
iteration: 422/19467
iteration: 423/19467
iteration: 424/19467
iteration: 425/19467
iteration: 426/19467
iteration: 427/19467
iteration: 428/19467
iteration: 429/19467
iteration: 430/19467
iteration: 431/19467
iteration: 432/19467
iteration: 433/19467
iteration: 434/19467
iteration: 435/19467
iteration: 436/19467
iteration: 437/19467
iteration: 438/19467
iteration: 439/19467
iteration: 440/19467
iteration: 441/19467
iteration: 442/19467
iteration: 443/19467
iteration: 44

iteration: 788/19467
iteration: 789/19467
iteration: 790/19467
iteration: 791/19467
iteration: 792/19467
iteration: 793/19467
iteration: 794/19467
iteration: 795/19467
iteration: 796/19467
iteration: 797/19467
iteration: 798/19467
iteration: 799/19467
iteration: 800/19467
iteration: 801/19467
iteration: 802/19467
iteration: 803/19467
iteration: 804/19467
iteration: 805/19467
iteration: 806/19467
iteration: 807/19467
iteration: 808/19467
iteration: 809/19467
iteration: 810/19467
iteration: 811/19467
iteration: 812/19467
iteration: 813/19467
iteration: 814/19467
iteration: 815/19467
iteration: 816/19467
iteration: 817/19467
iteration: 818/19467
iteration: 819/19467
iteration: 820/19467
iteration: 821/19467
iteration: 822/19467
iteration: 823/19467
iteration: 824/19467
iteration: 825/19467
iteration: 826/19467
iteration: 827/19467
iteration: 828/19467
iteration: 829/19467
iteration: 830/19467
iteration: 831/19467
iteration: 832/19467
iteration: 833/19467
iteration: 834/19467
iteration: 83

iteration: 1171/19467
iteration: 1172/19467
iteration: 1173/19467
iteration: 1174/19467
iteration: 1175/19467
iteration: 1176/19467
iteration: 1177/19467
iteration: 1178/19467
iteration: 1179/19467
iteration: 1180/19467
iteration: 1181/19467
iteration: 1182/19467
iteration: 1183/19467
iteration: 1184/19467
iteration: 1185/19467
iteration: 1186/19467
iteration: 1187/19467
iteration: 1188/19467
iteration: 1189/19467
iteration: 1190/19467
iteration: 1191/19467
iteration: 1192/19467
iteration: 1193/19467
iteration: 1194/19467
iteration: 1195/19467
iteration: 1196/19467
iteration: 1197/19467
iteration: 1198/19467
iteration: 1199/19467
iteration: 1200/19467
iteration: 1201/19467
iteration: 1202/19467
iteration: 1203/19467
iteration: 1204/19467
iteration: 1205/19467
iteration: 1206/19467
iteration: 1207/19467
iteration: 1208/19467
iteration: 1209/19467
iteration: 1210/19467
iteration: 1211/19467
iteration: 1212/19467
iteration: 1213/19467
iteration: 1214/19467
iteration: 1215/19467
iteration:

iteration: 1544/19467
iteration: 1545/19467
iteration: 1546/19467
iteration: 1547/19467
iteration: 1548/19467
iteration: 1549/19467
iteration: 1550/19467
iteration: 1551/19467
iteration: 1552/19467
iteration: 1553/19467
iteration: 1554/19467
iteration: 1555/19467
iteration: 1556/19467
iteration: 1557/19467
iteration: 1558/19467
iteration: 1559/19467
iteration: 1560/19467
iteration: 1561/19467
iteration: 1562/19467
iteration: 1563/19467
iteration: 1564/19467
iteration: 1565/19467
iteration: 1566/19467
iteration: 1567/19467
iteration: 1568/19467
iteration: 1569/19467
iteration: 1570/19467
iteration: 1571/19467
iteration: 1572/19467
iteration: 1573/19467
iteration: 1574/19467
iteration: 1575/19467
iteration: 1576/19467
iteration: 1577/19467
iteration: 1578/19467
iteration: 1579/19467
iteration: 1580/19467
iteration: 1581/19467
iteration: 1582/19467
iteration: 1583/19467
iteration: 1584/19467
iteration: 1585/19467
iteration: 1586/19467
iteration: 1587/19467
iteration: 1588/19467
iteration:

...

iteration: 17997/19467
iteration: 17998/19467
iteration: 17999/19467
iteration: 18000/19467
iteration: 18001/19467
iteration: 18002/19467
iteration: 18003/19467
iteration: 18004/19467
iteration: 18005/19467
iteration: 18006/19467
iteration: 18007/19467
iteration: 18008/19467
iteration: 18009/19467
iteration: 18010/19467
iteration: 18011/19467
iteration: 18012/19467
iteration: 18013/19467
iteration: 18014/19467
iteration: 18015/19467
iteration: 18016/19467
iteration: 18017/19467
iteration: 18018/19467
iteration: 18019/19467
iteration: 18020/19467
iteration: 18021/19467
iteration: 18022/19467
iteration: 18023/19467
iteration: 18024/19467
iteration: 18025/19467
iteration: 18026/19467
iteration: 18027/19467
iteration: 18028/19467
iteration: 18029/19467
iteration: 18030/19467
iteration: 18031/19467
iteration: 18032/19467
iteration: 18033/19467
iteration: 18034/19467
iteration: 18035/19467
iteration: 18036/19467
iteration: 18037/19467
iteration: 18038/19467
iteration: 18039/19467
iteration: 

iteration: 18354/19467
iteration: 18355/19467
iteration: 18356/19467
iteration: 18357/19467
iteration: 18358/19467
iteration: 18359/19467
iteration: 18360/19467
iteration: 18361/19467
iteration: 18362/19467
iteration: 18363/19467
iteration: 18364/19467
iteration: 18365/19467
iteration: 18366/19467
iteration: 18367/19467
iteration: 18368/19467
iteration: 18369/19467
iteration: 18370/19467
iteration: 18371/19467
iteration: 18372/19467
iteration: 18373/19467
iteration: 18374/19467
iteration: 18375/19467
iteration: 18376/19467
iteration: 18377/19467
iteration: 18378/19467
iteration: 18379/19467
iteration: 18380/19467
iteration: 18381/19467
iteration: 18382/19467
iteration: 18383/19467
iteration: 18384/19467
iteration: 18385/19467
iteration: 18386/19467
iteration: 18387/19467
iteration: 18388/19467
iteration: 18389/19467
iteration: 18390/19467
iteration: 18391/19467
iteration: 18392/19467
iteration: 18393/19467
iteration: 18394/19467
iteration: 18395/19467
iteration: 18396/19467
iteration: 

iteration: 18711/19467
iteration: 18712/19467
iteration: 18713/19467
iteration: 18714/19467
iteration: 18715/19467
iteration: 18716/19467
iteration: 18717/19467
iteration: 18718/19467
iteration: 18719/19467
iteration: 18720/19467
iteration: 18721/19467
iteration: 18722/19467
iteration: 18723/19467
iteration: 18724/19467
iteration: 18725/19467
iteration: 18726/19467
iteration: 18727/19467
iteration: 18728/19467
iteration: 18729/19467
iteration: 18730/19467
iteration: 18731/19467
iteration: 18732/19467
iteration: 18733/19467
iteration: 18734/19467
iteration: 18735/19467
iteration: 18736/19467
iteration: 18737/19467
iteration: 18738/19467
iteration: 18739/19467
iteration: 18740/19467
iteration: 18741/19467
iteration: 18742/19467
iteration: 18743/19467
iteration: 18744/19467
iteration: 18745/19467
iteration: 18746/19467
iteration: 18747/19467
iteration: 18748/19467
iteration: 18749/19467
iteration: 18750/19467
iteration: 18751/19467
iteration: 18752/19467
iteration: 18753/19467
iteration: 

iteration: 19070/19467
iteration: 19071/19467
iteration: 19072/19467
iteration: 19073/19467
iteration: 19074/19467
iteration: 19075/19467
iteration: 19076/19467
iteration: 19077/19467
iteration: 19078/19467
iteration: 19079/19467
iteration: 19080/19467
iteration: 19081/19467
iteration: 19082/19467
iteration: 19083/19467
iteration: 19084/19467
iteration: 19085/19467
iteration: 19086/19467
iteration: 19087/19467
iteration: 19088/19467
iteration: 19089/19467
iteration: 19090/19467
iteration: 19091/19467
iteration: 19092/19467
iteration: 19093/19467
iteration: 19094/19467
iteration: 19095/19467
iteration: 19096/19467
iteration: 19097/19467
iteration: 19098/19467
iteration: 19099/19467
iteration: 19100/19467
iteration: 19101/19467
iteration: 19102/19467
iteration: 19103/19467
iteration: 19104/19467
iteration: 19105/19467
iteration: 19106/19467
iteration: 19107/19467
iteration: 19108/19467
iteration: 19109/19467
iteration: 19110/19467
iteration: 19111/19467
iteration: 19112/19467
iteration: 

iteration: 19428/19467
iteration: 19429/19467
iteration: 19430/19467
iteration: 19431/19467
iteration: 19432/19467
iteration: 19433/19467
iteration: 19434/19467
iteration: 19435/19467
iteration: 19436/19467
iteration: 19437/19467
iteration: 19438/19467
iteration: 19439/19467
iteration: 19440/19467
iteration: 19441/19467
iteration: 19442/19467
iteration: 19443/19467
iteration: 19444/19467
iteration: 19445/19467
iteration: 19446/19467
iteration: 19447/19467
iteration: 19448/19467
iteration: 19449/19467
iteration: 19450/19467
iteration: 19451/19467
iteration: 19452/19467
iteration: 19453/19467
iteration: 19454/19467
iteration: 19455/19467
iteration: 19456/19467
iteration: 19457/19467
iteration: 19458/19467
iteration: 19459/19467
iteration: 19460/19467
iteration: 19461/19467
iteration: 19462/19467
iteration: 19463/19467
iteration: 19464/19467
iteration: 19465/19467
iteration: 19466/19467


In [21]:
X_test = similarity_matrix(xt_test)

iteration: 0/8692
iteration: 1/8692
iteration: 2/8692
iteration: 3/8692
iteration: 4/8692
iteration: 5/8692
iteration: 6/8692
iteration: 7/8692
iteration: 8/8692
iteration: 9/8692
iteration: 10/8692
iteration: 11/8692
iteration: 12/8692
iteration: 13/8692
iteration: 14/8692
iteration: 15/8692
iteration: 16/8692
iteration: 17/8692
iteration: 18/8692
iteration: 19/8692
iteration: 20/8692
iteration: 21/8692
iteration: 22/8692
iteration: 23/8692
iteration: 24/8692
iteration: 25/8692
iteration: 26/8692
iteration: 27/8692
iteration: 28/8692
iteration: 29/8692
iteration: 30/8692
iteration: 31/8692
iteration: 32/8692
iteration: 33/8692
iteration: 34/8692
iteration: 35/8692
iteration: 36/8692
iteration: 37/8692
iteration: 38/8692
iteration: 39/8692
iteration: 40/8692
iteration: 41/8692
iteration: 42/8692
iteration: 43/8692
iteration: 44/8692
iteration: 45/8692
iteration: 46/8692
iteration: 47/8692
iteration: 48/8692
iteration: 49/8692
iteration: 50/8692
iteration: 51/8692
iteration: 52/8692
ite

iteration: 417/8692
iteration: 418/8692
iteration: 419/8692
iteration: 420/8692
iteration: 421/8692
iteration: 422/8692
iteration: 423/8692
iteration: 424/8692
iteration: 425/8692
iteration: 426/8692
iteration: 427/8692
iteration: 428/8692
iteration: 429/8692
iteration: 430/8692
iteration: 431/8692
iteration: 432/8692
iteration: 433/8692
iteration: 434/8692
iteration: 435/8692
iteration: 436/8692
iteration: 437/8692
iteration: 438/8692
iteration: 439/8692
iteration: 440/8692
iteration: 441/8692
iteration: 442/8692
iteration: 443/8692
iteration: 444/8692
iteration: 445/8692
iteration: 446/8692
iteration: 447/8692
iteration: 448/8692
iteration: 449/8692
iteration: 450/8692
iteration: 451/8692
iteration: 452/8692
iteration: 453/8692
iteration: 454/8692
iteration: 455/8692
iteration: 456/8692
iteration: 457/8692
iteration: 458/8692
iteration: 459/8692
iteration: 460/8692
iteration: 461/8692
iteration: 462/8692
iteration: 463/8692
iteration: 464/8692
iteration: 465/8692
iteration: 466/8692


iteration: 827/8692
iteration: 828/8692
iteration: 829/8692
iteration: 830/8692
iteration: 831/8692
iteration: 832/8692
iteration: 833/8692
iteration: 834/8692
iteration: 835/8692
iteration: 836/8692
iteration: 837/8692
iteration: 838/8692
iteration: 839/8692
iteration: 840/8692
iteration: 841/8692
iteration: 842/8692
iteration: 843/8692
iteration: 844/8692
iteration: 845/8692
iteration: 846/8692
iteration: 847/8692
iteration: 848/8692
iteration: 849/8692
iteration: 850/8692
iteration: 851/8692
iteration: 852/8692
iteration: 853/8692
iteration: 854/8692
iteration: 855/8692
iteration: 856/8692
iteration: 857/8692
iteration: 858/8692
iteration: 859/8692
iteration: 860/8692
iteration: 861/8692
iteration: 862/8692
iteration: 863/8692
iteration: 864/8692
iteration: 865/8692
iteration: 866/8692
iteration: 867/8692
iteration: 868/8692
iteration: 869/8692
iteration: 870/8692
iteration: 871/8692
iteration: 872/8692
iteration: 873/8692
iteration: 874/8692
iteration: 875/8692
iteration: 876/8692


iteration: 1226/8692
iteration: 1227/8692
iteration: 1228/8692
iteration: 1229/8692
iteration: 1230/8692
iteration: 1231/8692
iteration: 1232/8692
iteration: 1233/8692
iteration: 1234/8692
iteration: 1235/8692
iteration: 1236/8692
iteration: 1237/8692
iteration: 1238/8692
iteration: 1239/8692
iteration: 1240/8692
iteration: 1241/8692
iteration: 1242/8692
iteration: 1243/8692
iteration: 1244/8692
iteration: 1245/8692
iteration: 1246/8692
iteration: 1247/8692
iteration: 1248/8692
iteration: 1249/8692
iteration: 1250/8692
iteration: 1251/8692
iteration: 1252/8692
iteration: 1253/8692
iteration: 1254/8692
iteration: 1255/8692
iteration: 1256/8692
iteration: 1257/8692
iteration: 1258/8692
iteration: 1259/8692
iteration: 1260/8692
iteration: 1261/8692
iteration: 1262/8692
iteration: 1263/8692
iteration: 1264/8692
iteration: 1265/8692
iteration: 1266/8692
iteration: 1267/8692
iteration: 1268/8692
iteration: 1269/8692
iteration: 1270/8692
iteration: 1271/8692
iteration: 1272/8692
iteration: 12

iteration: 1618/8692
iteration: 1619/8692
iteration: 1620/8692
iteration: 1621/8692
iteration: 1622/8692
iteration: 1623/8692
iteration: 1624/8692
iteration: 1625/8692
iteration: 1626/8692
iteration: 1627/8692
iteration: 1628/8692
iteration: 1629/8692
iteration: 1630/8692
iteration: 1631/8692
iteration: 1632/8692
iteration: 1633/8692
iteration: 1634/8692
iteration: 1635/8692
iteration: 1636/8692
iteration: 1637/8692
iteration: 1638/8692
iteration: 1639/8692
iteration: 1640/8692
iteration: 1641/8692
iteration: 1642/8692
iteration: 1643/8692
iteration: 1644/8692
iteration: 1645/8692
iteration: 1646/8692
iteration: 1647/8692
iteration: 1648/8692
iteration: 1649/8692
iteration: 1650/8692
iteration: 1651/8692
iteration: 1652/8692
iteration: 1653/8692
iteration: 1654/8692
iteration: 1655/8692
iteration: 1656/8692
iteration: 1657/8692
iteration: 1658/8692
iteration: 1659/8692
iteration: 1660/8692
iteration: 1661/8692
iteration: 1662/8692
iteration: 1663/8692
iteration: 1664/8692
iteration: 16

...

iteration: 7100/8692
iteration: 7101/8692
iteration: 7102/8692
iteration: 7103/8692
iteration: 7104/8692
iteration: 7105/8692
iteration: 7106/8692
iteration: 7107/8692
iteration: 7108/8692
iteration: 7109/8692
iteration: 7110/8692
iteration: 7111/8692
iteration: 7112/8692
iteration: 7113/8692
iteration: 7114/8692
iteration: 7115/8692
iteration: 7116/8692
iteration: 7117/8692
iteration: 7118/8692
iteration: 7119/8692
iteration: 7120/8692
iteration: 7121/8692
iteration: 7122/8692
iteration: 7123/8692
iteration: 7124/8692
iteration: 7125/8692
iteration: 7126/8692
iteration: 7127/8692
iteration: 7128/8692
iteration: 7129/8692
iteration: 7130/8692
iteration: 7131/8692
iteration: 7132/8692
iteration: 7133/8692
iteration: 7134/8692
iteration: 7135/8692
iteration: 7136/8692
iteration: 7137/8692
iteration: 7138/8692
iteration: 7139/8692
iteration: 7140/8692
iteration: 7141/8692
iteration: 7142/8692
iteration: 7143/8692
iteration: 7144/8692
iteration: 7145/8692
iteration: 7146/8692
iteration: 71

iteration: 7491/8692
iteration: 7492/8692
iteration: 7493/8692
iteration: 7494/8692
iteration: 7495/8692
iteration: 7496/8692
iteration: 7497/8692
iteration: 7498/8692
iteration: 7499/8692
iteration: 7500/8692
iteration: 7501/8692
iteration: 7502/8692
iteration: 7503/8692
iteration: 7504/8692
iteration: 7505/8692
iteration: 7506/8692
iteration: 7507/8692
iteration: 7508/8692
iteration: 7509/8692
iteration: 7510/8692
iteration: 7511/8692
iteration: 7512/8692
iteration: 7513/8692
iteration: 7514/8692
iteration: 7515/8692
iteration: 7516/8692
iteration: 7517/8692
iteration: 7518/8692
iteration: 7519/8692
iteration: 7520/8692
iteration: 7521/8692
iteration: 7522/8692
iteration: 7523/8692
iteration: 7524/8692
iteration: 7525/8692
iteration: 7526/8692
iteration: 7527/8692
iteration: 7528/8692
iteration: 7529/8692
iteration: 7530/8692
iteration: 7531/8692
iteration: 7532/8692
iteration: 7533/8692
iteration: 7534/8692
iteration: 7535/8692
iteration: 7536/8692
iteration: 7537/8692
iteration: 75

iteration: 7882/8692
iteration: 7883/8692
iteration: 7884/8692
iteration: 7885/8692
iteration: 7886/8692
iteration: 7887/8692
iteration: 7888/8692
iteration: 7889/8692
iteration: 7890/8692
iteration: 7891/8692
iteration: 7892/8692
iteration: 7893/8692
iteration: 7894/8692
iteration: 7895/8692
iteration: 7896/8692
iteration: 7897/8692
iteration: 7898/8692
iteration: 7899/8692
iteration: 7900/8692
iteration: 7901/8692
iteration: 7902/8692
iteration: 7903/8692
iteration: 7904/8692
iteration: 7905/8692
iteration: 7906/8692
iteration: 7907/8692
iteration: 7908/8692
iteration: 7909/8692
iteration: 7910/8692
iteration: 7911/8692
iteration: 7912/8692
iteration: 7913/8692
iteration: 7914/8692
iteration: 7915/8692
iteration: 7916/8692
iteration: 7917/8692
iteration: 7918/8692
iteration: 7919/8692
iteration: 7920/8692
iteration: 7921/8692
iteration: 7922/8692
iteration: 7923/8692
iteration: 7924/8692
iteration: 7925/8692
iteration: 7926/8692
iteration: 7927/8692
iteration: 7928/8692
iteration: 79

iteration: 8274/8692
iteration: 8275/8692
iteration: 8276/8692
iteration: 8277/8692
iteration: 8278/8692
iteration: 8279/8692
iteration: 8280/8692
iteration: 8281/8692
iteration: 8282/8692
iteration: 8283/8692
iteration: 8284/8692
iteration: 8285/8692
iteration: 8286/8692
iteration: 8287/8692
iteration: 8288/8692
iteration: 8289/8692
iteration: 8290/8692
iteration: 8291/8692
iteration: 8292/8692
iteration: 8293/8692
iteration: 8294/8692
iteration: 8295/8692
iteration: 8296/8692
iteration: 8297/8692
iteration: 8298/8692
iteration: 8299/8692
iteration: 8300/8692
iteration: 8301/8692
iteration: 8302/8692
iteration: 8303/8692
iteration: 8304/8692
iteration: 8305/8692
iteration: 8306/8692
iteration: 8307/8692
iteration: 8308/8692
iteration: 8309/8692
iteration: 8310/8692
iteration: 8311/8692
iteration: 8312/8692
iteration: 8313/8692
iteration: 8314/8692
iteration: 8315/8692
iteration: 8316/8692
iteration: 8317/8692
iteration: 8318/8692
iteration: 8319/8692
iteration: 8320/8692
iteration: 83

iteration: 8666/8692
iteration: 8667/8692
iteration: 8668/8692
iteration: 8669/8692
iteration: 8670/8692
iteration: 8671/8692
iteration: 8672/8692
iteration: 8673/8692
iteration: 8674/8692
iteration: 8675/8692
iteration: 8676/8692
iteration: 8677/8692
iteration: 8678/8692
iteration: 8679/8692
iteration: 8680/8692
iteration: 8681/8692
iteration: 8682/8692
iteration: 8683/8692
iteration: 8684/8692
iteration: 8685/8692
iteration: 8686/8692
iteration: 8687/8692
iteration: 8688/8692
iteration: 8689/8692
iteration: 8690/8692
iteration: 8691/8692


In [22]:
from keras.optimizers import SGD

#Choose optimizer:
optim = optimizers.Adam(lr=0.00015)
#optim = SGD(lr=0.1)

# create NN for news clissification:
news_DNN = Sequential()
news_DNN.add(Dense(64, input_dim=512, activation = 'relu'))#,kernel_regularizer=l2(0.1)))
news_DNN.add(Dropout(0.25))
news_DNN.add(Dense(32, activation = 'relu'))#,kernel_regularizer=l2(0.1)))
news_DNN.add(Dropout(0.25))


# Output layer with multiclass activation function:
news_DNN.add(Dense(4,activation='softmax'))

# Compile model:
news_DNN.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['acc'])

W0105 16:15:51.134172 139981392242496 deprecation_wrapper.py:119] From /home/marcus/miniconda3/envs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0105 16:15:51.259937 139981392242496 deprecation_wrapper.py:119] From /home/marcus/miniconda3/envs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0105 16:15:51.298090 139981392242496 deprecation_wrapper.py:119] From /home/marcus/miniconda3/envs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0105 16:15:51.579355 139981392242496 deprecation_wrapper.py:119] From /home/marcus/miniconda3/envs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecate

In [23]:
m_h = news_DNN.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), batch_size=64, verbose=1)

W0105 16:15:52.268402 139981392242496 deprecation.py:323] From /home/marcus/miniconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 19467 samples, validate on 8692 samples
Epoch 1/100
19467/19467 [==============================] - 3s 155us/step - loss: 1.2994 - acc: 0.3905 - val_loss: 1.2423 - val_acc: 0.4135
Epoch 2/100
19467/19467 [==============================] - 2s 84us/step - loss: 1.1803 - acc: 0.5038 - val_loss: 1.1355 - val_acc: 0.5411
Epoch 3/100
19467/19467 [==============================] - 2s 85us/step - loss: 1.1031 - acc: 0.5482 - val_loss: 1.0806 - val_acc: 0.5620
Epoch 4/100
19467/19467 [==============================] - 2s 89us/step - loss: 1.0671 - acc: 0.5692 - val_loss: 1.0537 - val_acc: 0.5729
Epoch 5/100
19467/19467 [==============================] - 2s 85us/step - loss: 1.0448 - acc: 0.5784 - val_loss: 1.0394 - val_acc: 0.5772
Epoch 6/100
19467/19467 [==============================] - 2s 79us/step - loss: 1.0253 - acc: 0.5871 - val_loss: 1.0303 - val_acc: 0.5811
Epoch 7/100
19467/19467 [==============================] - 2s 77us/step - loss: 1.0151 - acc: 0.5958 - val_loss: 1.0232 - 

In [24]:
news_DNN.save('news_DNN_v1')

In [25]:
E_train.shape

NameError: name 'E_train' is not defined

In [ ]:
E_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_test